<a href="https://colab.research.google.com/github/rodrigo103/AprendizajeAutomatico/blob/main/TP1_clasificaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# Librerías de análisis de datos

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from pandas.api.types import CategoricalDtype

from pandas.plotting import scatter_matrix

from pprint import pprint

In [36]:
# Librerías de Machine Learning

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
from xgboost import XGBClassifier

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
df = pd.read_csv("drive/MyDrive/AprendizajeAutomatico/OnlineNewsPopularity.csv", sep=', ')

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Corriendo el siguiente código puedo generar la columna de los target para la claficación.

In [39]:
def get_label(share):
  if share <= 870:
    return 'Very Poor'
  elif share <= 1200:
    return 'Poor'
  elif share <= 1800:
    return 'Average'
  elif share <= 3400:
    return 'Good'
  else:
    return 'Very Good'

df['shares_clasif'] = df['shares'].apply(get_label)
df

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,shares_clasif
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.70,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,Very Poor
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.70,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,Very Poor
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,1.00,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,Average
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.80,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,Poor
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,1.00,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,Very Poor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,http://mashable.com/2014/12/27/samsung-app-aut...,8.0,11.0,346.0,0.529052,1.0,0.684783,9.0,7.0,1.0,...,0.75,-0.260000,-0.500,-0.125000,0.100000,0.000000,0.400000,0.000000,1800,Average
39640,http://mashable.com/2014/12/27/seth-rogen-jame...,8.0,12.0,328.0,0.696296,1.0,0.885057,9.0,7.0,3.0,...,0.70,-0.211111,-0.400,-0.100000,0.300000,1.000000,0.200000,1.000000,1900,Good
39641,http://mashable.com/2014/12/27/son-pays-off-mo...,8.0,10.0,442.0,0.516355,1.0,0.644128,24.0,1.0,12.0,...,0.50,-0.356439,-0.800,-0.166667,0.454545,0.136364,0.045455,0.136364,1900,Good
39642,http://mashable.com/2014/12/27/ukraine-blasts/,8.0,6.0,682.0,0.539493,1.0,0.692661,10.0,1.0,1.0,...,0.50,-0.205246,-0.500,-0.012500,0.000000,0.000000,0.500000,0.000000,1100,Poor


# Clasificación
Entreno los modelos de clasificación usando la misma configuración que para regresión. Los modelos son los mismos pero en su versión para clasificación.

In [40]:
features = [
    'timedelta', 
    # Words
    'n_tokens_title', 'n_tokens_content', 
    'average_token_length', 
    'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
    'num_hrefs', 'num_self_hrefs',
    # Digital Media
    'num_imgs', 'num_videos', 
    # Links
    'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
    # Time
    'weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday', 
    'is_weekend', 
    # Keywords
    'num_keywords', 
    'kw_min_min', 'kw_max_min', 'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
    'data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 
    # Natural Language Processing
    'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 
    'global_subjectivity', 'global_sentiment_polarity', 
    'global_rate_positive_words', 'global_rate_negative_words', 
    'rate_positive_words', 'rate_negative_words', 
    'avg_positive_polarity', 'min_positive_polarity', 'max_positive_polarity', 
    'avg_negative_polarity', 'min_negative_polarity', 'max_negative_polarity', 
    'title_subjectivity', 'title_sentiment_polarity', 'abs_title_subjectivity', 'abs_title_sentiment_polarity'
]

# Se separa el dataframe en features (X) y target (y)
X = df[features]
y = df.shares_clasif


# Lo primero que hay que hacer es separar un conjunto de datos para test
trainval_X, test_X, trainval_y, test_y = train_test_split(X, y, random_state = 0, test_size = 0.3)

# Ahora separo el conjunto de datos restante (trainval) en entrenamiento (train) y validación (val)
train_X, val_X, train_y, val_y = train_test_split(trainval_X, trainval_y, random_state = 0)

Relizo un entrenamiento con varios modelos sobre todos los features para establecer una linea de base

In [41]:
scoring = {
    'accuracy'
}

preprocessor = ColumnTransformer(
    [
      ('power_transformer', PowerTransformer(), 
        [
          'num_hrefs',
          'num_self_hrefs',
          'n_tokens_content',
          'self_reference_min_shares', 
          'self_reference_max_shares', 
          'self_reference_avg_sharess',
        ]
      ),
      ('standard_scaler', StandardScaler(), 
        [
          # 'n_tokens_title',
          'global_subjectivity', 
          'global_sentiment_polarity',
          'global_rate_positive_words', 
          'global_rate_negative_words',
          'rate_positive_words',
          'rate_negative_words',
          'avg_positive_polarity'
        ]
      ),
    ], remainder='passthrough')

In [42]:
params = {
  'model__max_leaf_nodes': [None, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 500, 5000],
  'model__random_state': [1]
}

model = DecisionTreeClassifier()

my_pipeline = Pipeline(steps=[ 
    ('standard_scaler', preprocessor),
    ('model', model)])

grid_pipeline = GridSearchCV(my_pipeline, 
                              params, 
                              cv = 5, 
                              scoring = scoring,
                              refit='accuracy',
                              return_train_score=True)

grid_pipeline.fit(train_X, train_y);


# Muestro resultados
print(grid_pipeline.cv_results_.keys())

print(grid_pipeline.cv_results_['param_model__max_leaf_nodes'].data)

print(grid_pipeline.cv_results_["mean_test_accuracy"][grid_pipeline.best_index_])
print(grid_pipeline.cv_results_["std_test_accuracy"][grid_pipeline.best_index_])

best_params_decision_tree = grid_pipeline.best_params_
print(f'Best parameters {best_params_decision_tree}')
print(f'Best score {grid_pipeline.best_score_}')

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_model__max_leaf_nodes', 'param_model__random_state', 'params', 'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy', 'split4_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy', 'split0_train_accuracy', 'split1_train_accuracy', 'split2_train_accuracy', 'split3_train_accuracy', 'split4_train_accuracy', 'mean_train_accuracy', 'std_train_accuracy'])
[None 2 3 4 5 6 7 8 9 10 20 30 40 50 500 5000]
0.30948474830844896
0.0038078269036087888
Best parameters {'model__max_leaf_nodes': 30, 'model__random_state': 1}
Best score 0.30948474830844896


In [43]:
params = {
  'model__n_estimators': [
    3,
    # 5, 10, 15, 
    # 50, 100, 200
  ],
  # 'model__max_depth': [7],
  'model__random_state': [1]
}

model = RandomForestClassifier()

my_pipeline = Pipeline(steps=[ 
    ('standard_scaler', preprocessor),
    ('model', model)])

grid_pipeline = GridSearchCV(my_pipeline, 
                              params, 
                              cv = 5, 
                              scoring = scoring,
                              refit='accuracy',
                              return_train_score=True)

grid_pipeline.fit(train_X, train_y);


# Muestro resultados
print(grid_pipeline.cv_results_.keys())

print(grid_pipeline.cv_results_['param_model__n_estimators'].data)

print(grid_pipeline.cv_results_["mean_test_accuracy"][grid_pipeline.best_index_])
print(grid_pipeline.cv_results_["std_test_accuracy"][grid_pipeline.best_index_])

best_params_random_forest = grid_pipeline.best_params_
print(f'Best parameters {best_params_random_forest}')
print(f'Best score {grid_pipeline.best_score_}')

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_model__n_estimators', 'param_model__random_state', 'params', 'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy', 'split4_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy', 'split0_train_accuracy', 'split1_train_accuracy', 'split2_train_accuracy', 'split3_train_accuracy', 'split4_train_accuracy', 'mean_train_accuracy', 'std_train_accuracy'])
[3]
0.2394291926438755
0.007295453915071603
Best parameters {'model__n_estimators': 3, 'model__random_state': 1}
Best score 0.2394291926438755


In [44]:
params = {
  'model__n_estimators': [
     5, 10, 15, 
      # 20, 25, 50, 100, 
      # 500, 
      # 1000
      ],
  'model__learning_rate': [0.01, 0.05, 0.1],
  'model__early_stopping_rounds':[5],
  'model__random_state': [1]
}

model = XGBClassifier()

my_pipeline = Pipeline(steps=[ 
    ('standard_scaler', preprocessor),
    ('model', model)])

grid_pipeline = GridSearchCV(my_pipeline, 
                              params, 
                              cv = 5, 
                              scoring = scoring,
                              refit='accuracy',
                              return_train_score=True)

grid_pipeline.fit(train_X, train_y);


# Muestro resultados
print(grid_pipeline.cv_results_.keys())

print(grid_pipeline.cv_results_['param_model__n_estimators'].data)

print(grid_pipeline.cv_results_["mean_test_accuracy"][grid_pipeline.best_index_])
print(grid_pipeline.cv_results_["std_test_accuracy"][grid_pipeline.best_index_])

best_params_xgb = grid_pipeline.best_params_
print(f'Best parameters {best_params_xgb}')
print(f'Best score {grid_pipeline.best_score_}')

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_model__early_stopping_rounds', 'param_model__learning_rate', 'param_model__n_estimators', 'param_model__random_state', 'params', 'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy', 'split4_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy', 'split0_train_accuracy', 'split1_train_accuracy', 'split2_train_accuracy', 'split3_train_accuracy', 'split4_train_accuracy', 'mean_train_accuracy', 'std_train_accuracy'])
[5 10 15 5 10 15 5 10 15]
0.31419356097277185
0.005251626023852254
Best parameters {'model__early_stopping_rounds': 5, 'model__learning_rate': 0.1, 'model__n_estimators': 15, 'model__random_state': 1}
Best score 0.31419356097277185


El rendimiento del mejor modelo tomando en cuenta la variable Accuracy antes de recortar las variables fue con XGBClassifier y fue de  
Accuracy: 0.31419356097277185  
con los parámetros:  
'model__early_stopping_rounds': 5, 
'model__learning_rate': 0.1, 
'model__n_estimators': 15, 
'model__random_state': 1


## Ahora voy a hacer un último fit y predict con el conjunto de test para calcular las metricas finales.

In [45]:
print(best_params_xgb)

my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBClassifier(
        n_estimators=15,
        learning_rate=0.1,
        random_state=1,
        ))
  ])

my_pipeline.fit(trainval_X, trainval_y)

preds = my_pipeline.predict(test_X)

print(f"Accuracy:\n{accuracy_score(test_y, preds)}" )

confusion = confusion_matrix(test_y, preds)
print(f"Confusion matrix:\n{confusion}")

print(f"\nNormalized confusion matrix:")
for row in confusion:
    print(row / row.sum())

print(f"\nProbs:")
probs = my_pipeline.predict_proba(test_X)
print(probs)

print(f"\nClassification report:\n{classification_report(test_y, preds)}")

{'model__early_stopping_rounds': 5, 'model__learning_rate': 0.1, 'model__n_estimators': 15, 'model__random_state': 1}
Accuracy:
0.3073818732133849
Confusion matrix:
[[ 136  359  608  675  498]
 [ 133  431  442  793  405]
 [ 118  256  788  612  876]
 [  96  375  337 1162  342]
 [  64  138  639  472 1139]]

Normalized confusion matrix:
[0.05975395 0.15773286 0.26713533 0.29657293 0.21880492]
[0.06034483 0.19555354 0.20054446 0.35980036 0.18375681]
[0.0445283  0.09660377 0.29735849 0.2309434  0.33056604]
[0.04152249 0.16219723 0.14576125 0.50259516 0.14792388]
[0.02610114 0.05628059 0.26060359 0.19249592 0.46451876]

Probs:
[[0.19988205 0.15450029 0.27037132 0.1357974  0.23944893]
 [0.20593055 0.20782182 0.22303702 0.16227145 0.2009392 ]
 [0.22262678 0.2119362  0.20416914 0.1733229  0.18794498]
 ...
 [0.16806817 0.19594984 0.19151081 0.2508479  0.19362323]
 [0.1872344  0.15279195 0.25797915 0.16498978 0.23700468]
 [0.1777406  0.15457109 0.25210765 0.16322768 0.25235298]]

Classification r

El resultado final fue:  
Accuracy: 0.3073818732133849

## Vuelvo a correr los modelos despreciando algunos features de acuerdo al análisis EDA

In [46]:
features = [
    # 'timedelta', 
    # Words
    'n_tokens_title', 'n_tokens_content', 
    # 'average_token_length', 
    # 'n_unique_tokens', 'n_non_stop_words', 'n_non_stop_unique_tokens',
    'num_hrefs', 'num_self_hrefs',
    # Digital Media
    # 'num_imgs', 'num_videos', 
    # Links
    'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
    # Time
    # 'weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday', 'weekday_is_sunday', 
    # 'is_weekend', 
    # Keywords
    'num_keywords', 
    'kw_min_min', 'kw_max_min', 'kw_avg_min', 'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
    'data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 
    # Natural Language Processing
    'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 
    'global_subjectivity', 'global_sentiment_polarity', 
    'global_rate_positive_words', 'global_rate_negative_words', 
    'rate_positive_words', 'rate_negative_words', 
    'avg_positive_polarity', 'min_positive_polarity', 'max_positive_polarity', 
    'avg_negative_polarity', 'min_negative_polarity', 'max_negative_polarity', 
    'title_subjectivity', 'title_sentiment_polarity', 'abs_title_subjectivity', 'abs_title_sentiment_polarity'
]

# Se separa el dataframe en features (X) y target (y)
X = df[features]
y = df.shares_clasif


# Lo primero que hay que hacer es separar un conjunto de datos para test
trainval_X, test_X, trainval_y, test_y = train_test_split(X, y, random_state = 0, test_size = 0.3)

# Ahora separo el conjunto de datos restante (trainval) en entrenamiento (train) y validación (val)
train_X, val_X, train_y, val_y = train_test_split(trainval_X, trainval_y, random_state = 0)

Vuelvo a probar XGBRegressor con menos features


In [47]:
params = {
  'model__n_estimators': [
     5, 10, 15, 
      # 20, 25, 50, 100, 
      # 500, 
      # 1000
      ],
  'model__learning_rate': [0.01, 0.05, 0.1],
  'model__early_stopping_rounds':[5],
  'model__random_state': [1]
}

model = XGBClassifier()

my_pipeline = Pipeline(steps=[ 
    ('standard_scaler', preprocessor),
    ('model', model)])

grid_pipeline = GridSearchCV(my_pipeline, 
                              params, 
                              cv = 5, 
                              scoring = scoring,
                              refit='accuracy',
                              return_train_score=True)

grid_pipeline.fit(train_X, train_y);


# Muestro resultados
print(grid_pipeline.cv_results_.keys())

print(grid_pipeline.cv_results_['param_model__n_estimators'].data)

print(grid_pipeline.cv_results_["mean_test_accuracy"][grid_pipeline.best_index_])
print(grid_pipeline.cv_results_["std_test_accuracy"][grid_pipeline.best_index_])

best_params_xgb = grid_pipeline.best_params_
print(f'Best parameters {best_params_xgb}')
print(f'Best score {grid_pipeline.best_score_}')

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_model__early_stopping_rounds', 'param_model__learning_rate', 'param_model__n_estimators', 'param_model__random_state', 'params', 'split0_test_accuracy', 'split1_test_accuracy', 'split2_test_accuracy', 'split3_test_accuracy', 'split4_test_accuracy', 'mean_test_accuracy', 'std_test_accuracy', 'rank_test_accuracy', 'split0_train_accuracy', 'split1_train_accuracy', 'split2_train_accuracy', 'split3_train_accuracy', 'split4_train_accuracy', 'mean_train_accuracy', 'std_train_accuracy'])
[5 10 15 5 10 15 5 10 15]
0.30146099543090477
0.004507725356524182
Best parameters {'model__early_stopping_rounds': 5, 'model__learning_rate': 0.1, 'model__n_estimators': 15, 'model__random_state': 1}
Best score 0.30146099543090477


In [48]:
print(best_params_xgb)

my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBClassifier(
        n_estimators=15,
        learning_rate=0.1,
        random_state=1,
        ))
  ])

my_pipeline.fit(trainval_X, trainval_y)

preds = my_pipeline.predict(test_X)

print(f"Accuracy:\n{accuracy_score(test_y, preds)}" )

confusion = confusion_matrix(test_y, preds)
print(f"Confusion matrix:\n{confusion}")

print(f"\nNormalized confusion matrix:")
for row in confusion:
    print(row / row.sum())

print(f"\nProbs:")
probs = my_pipeline.predict_proba(test_X)
print(probs)

print(f"\nClassification report:\n{classification_report(test_y, preds)}")

{'model__early_stopping_rounds': 5, 'model__learning_rate': 0.1, 'model__n_estimators': 15, 'model__random_state': 1}
Accuracy:
0.29418194047418866
Confusion matrix:
[[ 102  358  680  679  457]
 [ 116  418  511  795  364]
 [ 115  282  864  623  766]
 [  76  378  372 1185  301]
 [  76  159  804  483  930]]

Normalized confusion matrix:
[0.04481547 0.1572935  0.29876977 0.2983304  0.20079086]
[0.05263158 0.18965517 0.23185118 0.3607078  0.16515426]
[0.04339623 0.10641509 0.32603774 0.23509434 0.2890566 ]
[0.03287197 0.16349481 0.16089965 0.51254325 0.13019031]
[0.03099511 0.06484502 0.3278956  0.19698206 0.37928222]

Probs:
[[0.21127896 0.14684638 0.2782014  0.1351441  0.22852916]
 [0.2086667  0.22196604 0.22885388 0.1699966  0.1705168 ]
 [0.22812076 0.22383827 0.21650456 0.16906402 0.1624724 ]
 ...
 [0.17121758 0.19118682 0.20435676 0.25498888 0.17824998]
 [0.19131464 0.15382823 0.26440197 0.16720156 0.22325361]
 [0.18084662 0.16038592 0.25382316 0.16583082 0.2391135 ]]

Classification 

El resultado fue ligeramente peor que con todas las features.